------------------------------------------------------------------------------------------------------------------------------------

# This Example shows how to use advanced postprocessing functions to create plots using MatPlotLib without opening HFSS UI
-------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import time
import os
import datetime
import pathlib
import sys

local_path = os.path.abspath('')
module_path = pathlib.Path(local_path)
aedt_lib_path = module_path.parent.parent.parent
example_path =os.path.join(module_path.parent.parent,"Examples","Examples_Files")
sys.path.append(os.path.join(aedt_lib_path))
from pyaedt import generate_unique_name
project_dir = os.path.join(os.environ["TEMP"], generate_unique_name("Example"))
if not os.path.exists(project_dir): os.makedirs(project_dir)
print(project_dir)


C:\Users\mcapodif\AppData\Local\Temp\Example_JXC5GM


# Input Variables

In [2]:
input_path = os.path.join(aedt_lib_path,"pyaedt", "Examples_Files")
material_name = 'MaterialExport.csv'
component_properties =  'Tutorial Board Parts List.csv'
component_step = 'Tutorial Board.stp'
aedt_odb_project = 'SherkockTutorial.aedt'
aedt_odb_design_name = "PCB"
stackup_thickness = 2.11836
outline_polygon_name = "poly_14188"

# Import pyaedt and start AEDT

AEDT will run in nongraphical mode

In [3]:
from pyaedt import Icepak
from pyaedt import Desktop
d=Desktop("2021.1", NG=False)
start = time.time()
material_list = os.path.join(example_path, material_name)
component_list = os.path.join(example_path, component_properties)
validation=os.path.join(project_dir, "validation.log")
file_path=os.path.join(example_path, component_step)
project_name=os.path.join(project_dir, component_step[:-3]+"aedt")

Info: Using Windows TaskManager to Load processes
Info: Ansoft.ElectronicsDesktop.2021.1 Started with Process ID 28120
Info: pyaedt v0.5.0 started
Info: Started external COM connection with module pythonnet_v3
Info: Python version 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
Info: Exe path: C:\Anaconda3\python.exe


In [4]:
# Create an Icepak project and delete Region to improve performances

In [5]:
ipk = Icepak()
#Removing region and disabling autosave to speedup import
d.disable_autosave()
ipk.modeler.primitives.delete("Region")
component_name = "from_ODB"

Info: Added design 'Icepak_LO6' of type Icepak
Info: Design Loaded
Info: Successfully loaded project materials !
Info: Materials Loaded
Info: Deleted 1 Objects


# Import PCB from aedb file

In [6]:
odb_path = os.path.join(example_path, aedt_odb_project)
ipk.create_pcb_from_3dlayout(component_name, odb_path, aedt_odb_design_name,extenttype="Polygon",
                               outlinepolygon=outline_polygon_name)

Info: Closing the specified SherkockTutorial AEDT Project
Info: PCB Component Correctly created in Icepak


True

# create an offset Coordinate system to match odb++ with sherlock step file

In [7]:
ipk.modeler.coordinate_system.create([0,0,stackup_thickness/2],view="XY")

# import cad

In [8]:
ipk.modeler.import_3d_cad(file_path, refresh_all_ids=False)
#save cad and refresh properties from aedt file parsing
ipk.save_project(project_name, refresh_obj_ids_after_save=True)

Info: Step file C:\GIT\repos\pyaedt\pyaedt\Examples\Examples_Files\Tutorial Board.stp imported
Info: Saving Project138 Project


True

# removing pcb objects

In [9]:
ipk.modeler.primitives.delete_objects_containing("pcb", False)

Info: Deleted 1 Objects
Info: Deleted 1 objects


True

# Creating Region

In [10]:
ipk.modeler.create_air_region(*[20,20,300,20,20,300])

6

# assigning Materials


In [11]:
ipk.assignmaterial_from_sherlock_files(component_list, material_list)

Info: Adding New Material material to Project Library: alumina
Info: Material added. Please edit it to update in Desktop
Info: Assign Material alumina to object COMP_R1,COMP_R2,COMP_R3,COMP_R4,COMP_R5,COMP_R6,COMP_R7,COMP_R8,COMP_R9,COMP_R10,COMP_R11,COMP_R12,COMP_R13,COMP_R14,COMP_R15,COMP_R16,COMP_R17,COMP_R18,COMP_R19,COMP_R20,COMP_R21,COMP_R22,COMP_R23,COMP_R24,COMP_R25,...
Info: Assign Material aluminum to object COMP_C21,COMP_C22,COMP_C23,COMP_C24,COMP_C25,COMP_C26,COMP_C27,COMP_C28,COMP_C29,COMP_C30
Info: Adding New Material material to Project Library: bariumtitanate
Info: Material added. Please edit it to update in Desktop
Info: Assign Material bariumtitanate to object COMP_C11,COMP_C12,COMP_C13,COMP_C14,COMP_C15,COMP_C16,COMP_C17,COMP_C18,COMP_C19,COMP_C20
Info: Assign Material copper to object COMP_J1
Info: Adding New Material material to Project Library: epoxyencapsulant
Info: Material added. Please edit it to update in Desktop
Info: Assign Material epoxyencapsulant to obje

True

# Deleting Object with no material Assignment

In [12]:
#ipk.create_source_blocks_from_list([["COMP_R3",1,2,3],["COMP_R4",4,5,6]], False)


In [13]:
#ipk.create_network_blocks([["COMP_R1",20,10,3],["COMP_R2",4,10,3]],ipk.GravityDirection.ZNeg,1.05918,False)

In [14]:
no_material_objs = ipk.modeler.primitives.get_objects_by_material("")
ipk.modeler.primitives.delete(no_material_objs)
#ipk.modeler.set_object_model_state(all_objs, False)
ipk.save_project()

Info: Deleted 0 Objects
Info: Saving Tutorial Board Project


True

# Assign Power to Component Blocks

In [15]:
all_objects = ipk.modeler.primitives.get_all_objects_names()
# Assign Power blocks
total_power = ipk.assign_block_from_sherlock_file(component_list)

Info: Blocks inserted with total power 34.2000000000002W


# Setup and Boundaries

In [16]:
ipk.mesh.automatic_mesh_pcb(4)

setup1 = ipk.create_setup()
setup1.props["Solution Initialization - Y Velocity"] =  "1m_per_sec"
setup1.props["Radiation Model"] ="Discrete Ordinates Model"
setup1.props["Include Gravity"] =True
setup1.props["Secondary Gradient"] =True
setup1.update()
ipk.assign_openings(ipk.modeler.primitives.get_object_faces("Region"))

Info: Face List boundary_faces created
Info: Opening Assigned


# Check for intersection using Validation and fix it by assigning Priorities

In [17]:
ipk.assign_priority_on_intersections()
# Saving and closing
ipk.save_project()

Info: Saving Tutorial Board Project
0 Intersections have been found. Applying Priorities
Info: Saving Tutorial Board Project


True

In [18]:
end = time.time()-start
ipk.close_desktop()
print("Elapsed time: {}".format(datetime.timedelta(seconds=end)))

Elapsed time: 0:00:44.151753


In [19]:
d.force_close_desktop()

AttributeError: module '__main__' has no attribute 'oDesktop'